In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 1.9 MB/s 
     |████████████████████████████████| 140 kB 46.5 MB/s 
     |████████████████████████████████| 358 kB 41.2 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ge

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


## `news_scraping()` : 뉴스 스크래핑 함수

In [3]:
def news_scraping(news_url, wd):

  press = wd.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[1]').get_attribute('title')
  title = wd.find_element(By.XPATH, '//*[@id="ct"]/div[1]/div[2]/h2').text
  datetime = wd.find_element(By.XPATH,'//*[@id="ct"]/div[1]/div[3]/div[1]/div[1]/span').text
  article = wd.find_element(By.XPATH,'//*[@id="dic_area"]').text
  article = article.replace("\n", "")
  article = article.replace("\t", "")

  useful = wd.find_element(By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[1]/a/span[2]').text
  wow = wd.find_element(By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[2]/a/span[2]').text
  touched = wd.find_element(By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[3]/a/span[2]').text
  analytical = wd.find_element(By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[4]/a/span[2]').text
  recommend = wd.find_element(By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[5]/a/span[2]').text
  try:
    recommend1 = wd.find_element(By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[6]/a/span[2]').text
  except:  
    print("recommend : ")

  print("뉴스:", [title, press, datetime, article, useful, wow, touched, analytical, recommend, news_url])

  return [title, press, datetime, article, useful, wow, touched, analytical, recommend, news_url]

## `comments_scraping`: 뉴스 댓글 스크래핑

In [4]:
def comments_scraping(news_url, wd):

  try:
    btn_view = wd.find_element(By.CLASS_NAME, 'u_cbox_btn_view_comment').click()
    print("[댓글 더보기]", end="")
    time.sleep(1)

    wd.find_element(By.CLASS_NAME, 'u_cbox_btn_more').click()
    print("[더보기]", end="")
    time.sleep(1)
  except:
    pass
  
  


## `scraping()` : 스크래핑 함수

In [5]:
def scraping():
  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)

  news_idx = 0
  news_df = pd.DataFrame(columns=("Title", "Press", "DateTime", "Article", "Useful", "Wow", "Touched", "Analytical", "Recommend", "News_Url"))

  news_url = 'https://n.news.naver.com/mnews/article/025/0003223485?sid=102'
  wd.get(news_url)

  news_df.loc[news_idx] = news_scraping(news_url, wd)
  news_idx += 1

  comments_scraping(news_url)

  wd.close()

  return news_df


In [ ]:
news_df = scraping()

recommend : 
뉴스: ['경찰, 성남FC 의혹 이재명에 ‘뇌물 혐의’ 검찰 송치', '중앙일보', '2022.09.14. 오전 12:31', '더불어민주당 이재명 대표(왼쪽)와 김태년 민생경제위기대책위원회 위원장이 13일 서울 여의도 국회에서 열린 민생경제위기대책위원회 출범식 및 기자간담회에서 대화하고 있다. 김경록 기자‘성남FC 후원금 의혹’을 수사해 온 경기남부경찰청(이하 경기남부청)이 이재명 더불어민주당 대표에게 제3자 뇌물제공 혐의가 인정된다는 보완수사 결과를 13일 검찰에 통보했다. 지난해 9월 경기도 분당경찰서가 내렸던 불송치 결정을 뒤집은 것이다. 경기남부청은 두산건설이 성남FC에 낸 50억원대 후원금이 성남시가 분당구 정자동 토지의 용도를 변경해 준 것의 대가라고 판단했다.경기남부청 반부패·경제범죄수사대는 이날 보완수사 결과를 검찰에 통보하고, 당시 주도적으로 실무를 담당한 성남시 공무원 1명도 같은 혐의의 공동정범으로 입건해 검찰에 송치했다. 당시 두산건설 대표이사 이모씨에 대해선 형법상 뇌물공여 혐의를 적용해 검찰에 넘겼다.이 대표가 성남시장으로 재직하던 시절 이 대표가 구단주이던 성남FC는 2014~2016년 두산건설에서 56억3000만원의 광고 후원금을 유치했고, 성남시는 2015년 7월 두산그룹이 소유한 분당구 정자동 병원 부지 3000여 평을 상업용지로 용도변경을 해줬다. 용적률과 건축 규모, 연면적 등을 3배가량 늘려주고 전체 부지의 10%를 기부채납받는 조건이었다. 경찰은 이 두 가지 행위가 대가 관계에 있다고 파악한 것이다. 두산이 1991년 72억원에 샀던 이 부지의 현재 평가가치는 1조원 이상인 것으로 알려져 있다.경찰의 수사는 2018년 6월 지방선거 과정에서 바른미래당이 “성남시가 용지 변경으로 두산 측에 막대한 이익을 줬다”며 고발장을 제출하면서 시작됐다. 의혹이 불거지자 성남시와 두산건설은 대가성을 부인했다. 이 대표도 지난해 8월 자신의 페이스북에 “성남시 소유인 성남FC가 용도변경 조건으로 광고비를 받았다고

In [ ]:
news_df

,Title,Press,DateTime,Article,Useful,Wow,Touched,Analytical,Recommend,News_Url
0,"경찰, 성남FC 의혹 이재명에 ‘뇌물 혐의’ 검찰 송치",중앙일보,2022.09.14. 오전 12:31,더불어민주당 이재명 대표(왼쪽)와 김태년 민생경제위기대책위원회 위원장이 13일 서울...,0,0,0,0,0,https://n.news.naver.com/mnews/article/025/000...


# `실종동물 스크래핑`

### [긴급] 실종 동물 스크래핑

In [6]:
def emergency_scraping(animal_url, wd):
  try:
    emergency_animals = []

    for t in range(1, 100):
      for i in range(1, 4):
        emergency_url = wd.find_element(By.XPATH, '/html/body/div[4]/div/div/div/div[%d]/div[1]/div[%d]/div/a' %(t,i) ).get_attribute('href')
        emergency_animals.append(emergency_url)

        
  except:
    pass
  return emergency_animals


## [실종] 실종 동물 스크래핑

In [44]:
def missing_animals_url(animal, wd):
  try: 
    missing_animals= []
    totalpages = wd.find_element(By.XPATH, '/html/body/div[6]/div/div/div/p').text
    page = totalpages.split(" ")
    last = int(page[6]) + 1
    
    for p in range(1, last):
      url = 'http://www.angel.or.kr/index.php?code=%s&page=%d' %(animal, p)
      wd.get(url)
      for t in range(1, 6):
        for i in range(1, 5):
          if(p==1):
            missing = wd.find_element(By.XPATH, '/html/body/div[5]/div/div/div/div[%d]/div[1]/div[%d]/h3' %(t,i) ).text
          else:
            missing = wd.find_element(By.XPATH, '/html/body/div[4]/div/div/div/div[%d]/div[1]/div[%d]/h3' %(t,i) ).text

          if("실종" in missing):  
            if(p==1):
              missing_url = wd.find_element(By.XPATH, '//html/body/div[5]/div/div/div/div[%d]/div[1]/div[%d]/div/a' %(t,i) ).get_attribute('href')
            else:
              missing_url = wd.find_element(By.XPATH, '//html/body/div[4]/div/div/div/div[%d]/div[1]/div[%d]/div/a' %(t,i) ).get_attribute('href')
            missing_animals.append(missing_url)
    
  except: 
    pass


  return missing_animals

## 실종 동물 상세 정보

In [45]:
def dog_info_scraping(animal_info_url, wd):

  animal = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[1]/td[2]').text
  info = animal.split('/')
  find = info[len(info)-4]
  gender = info[len(info)-3]
  age = info[len(info)-2]
  name = info[len(info)-1]
  missingDay = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[2]/td[2]').text
  location = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[3]/td[2]').text
  description = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/table/tbody/tr[5]/td[2]').text
  description = description.replace("\n", " ")
  description = description.replace("\t", " ")

  # 이미지 클릭해 이미지로 이동
  img_url = wd.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/div[3]/div[1]/div[1]/div/a').get_attribute('href')
  wd.get(img_url)
  img = wd.find_element(By.XPATH, '/html/body/img').get_attribute('src')


  return [find, gender, age, name, missingDay, location ,description, img]

## `URL_스크래핑` : 강아지

In [54]:
def dog_url_scraping():

  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)
  
  dogs_url = 'http://www.angel.or.kr/index.php?code=dog'
  wd.get(dogs_url)
  
  dogs_df = pd.DataFrame(columns=("Find", "Gender", "Age", "Name", "MissingDay", "Location", "Description", "IMG"))

  emergency_dogs = emergency_scraping(dogs_url, wd)
  
  dogs_idx = 0
  for dog_url in emergency_dogs:
    wd.get(dog_url)
    dogs_df.loc[dogs_idx] = dog_info_scraping(dog_url, wd)
    dogs_idx += 1

  print(dogs_idx)
  missing_dogs_url = missing_animals_url('dog', wd)
  print(missing_dogs_url)
  for dog_url in missing_dogs_url:
    wd.get(dog_url)
    dogs_df.loc[dogs_idx] = dog_info_scraping(dog_url, wd)
    dogs_idx += 1

  return dogs_df
  


In [55]:
dog_df = dog_url_scraping()

15
[]


In [51]:
dog_df

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,포메라니언,암컷,6살,이름(이브),2022-09-13,전라북도 김제시 용지면 용지면사무소 근처,"전체적으로 황색인데 가슴,배쪽은 흰털 사람 엄청 좋아함 눈 주위가 하얗고 아이라인이...",http://www.angel.or.kr/report/dog/upImg/166313...
1,이탈리언 그레이하운드,암컷,6살,이름(세모),2022-09-01,충청남도 아산시 아산 은행나무길 / 삽교회전교차로,얼룩무늬 그레이하운드과 휘펫 노락색옷입고있습니다 사슴같이 말랐습니다,http://www.angel.or.kr/report/dog/upImg/166313...
2,진돗개,암컷,1살미만,이름(알콩이),2022-09-13,경기도 남양주시 중흥s클래스 2206동 주차장,"겁많음, 성인남자트라우마, 소리예민, 큰움직임 예민",http://www.angel.or.kr/report/dog/upImg/166305...
3,믹스견(진도 믹스 아이보리색상),수컷,5살,이름(까불이),2022-9-10,경기도 고양시 일산동구 성석동 빅토리아 골프연습장 인근,몸 길이 80cm 중성화 안됨 심장사상충 약 복용으로 입과 코 주변이 하얀색임 목줄...,http://www.angel.or.kr/report/dog/upImg/166291...
4,믹스견(말티즈+시츄),암컷,9살,이름(코코),2022-9-2,경기도 수원시 팔달구 수원월드메르디앙- 수원월드컵 경기장,속눈썹이 엄청 길어요,http://www.angel.or.kr/report/dog/upImg/166226...
5,진돗개,암컷,3살,이름(미니),2022-09-02,서울특별시 강서구 우장산공원터널부근,베이지색깔 반접힌귀 앞다리 반 흰색 꼬리 끝 흰색,http://www.angel.or.kr/report/dog/upImg/166216...
6,기타견종(보더콜리 진도견 믹스),암컷,5살,이름(콜리),2022-9-2,충청북도 진천군 진천읍 신정리 문화3길 천호빌라 103동307호,겁이 많다 사람 경계많이함 크기는 중형견,http://www.angel.or.kr/report/dog/upImg/166212...
7,믹스견(비글 요크셔 혼종),수컷,6살,이름(은솔),2022-09-01,서울특별시 강서구 허준 박물관 인근,크림빛누렁이주둥이 검정/늘씬하고 길죽한 몸/겁이많아경계심이많음/최근미용빡빡이/독특한...,http://www.angel.or.kr/report/dog/upImg/166206...
8,셔틀랜드 쉽독,수컷,6살,이름(지니),2022-8-24,전라남도 고흥군 두원면 용반리 벧엘수양관 부근,소심하고 겁이 많아요.인식칩 목걸이 착용. 노랑색 오리발모양 펜던트 착용.전화번호 ...,http://www.angel.or.kr/report/dog/upImg/166183...
9,기타견종(그레이트피레니즈),수컷,1살,이름(조이),2022-8-20,경기도 양주시 회암동 옥정숲속어린이집 근처,하얀색 털갈이 중으로 모종이 풍성하지 않습니다. 몸무게 50kg 이상이고 내장인식칩...,http://www.angel.or.kr/report/dog/upImg/166120...


## `URL_스크래핑` : 고양이

In [48]:
def cat_url_scraping():
  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)
  cats_url = 'http://www.angel.or.kr/index.php?code=cat'
  
  wd.get(cats_url)

  cats_df = pd.DataFrame(columns=("Find", "Gender", "Age", "Name", "MissingDay", "Location", "Description", "IMG"))

  emergency_cats = emergency_scraping(cats_url, wd)

  cats_idx = 0
  for cat_url in emergency_cats:
    wd.get(cat_url)
    cats_df.loc[cats_idx] = dog_info_scraping(cat_url, wd)
    cats_idx += 1

  missing_cats_url = missing_animals_url('cat', wd)

  for cat_url in missing_cats_url:
    wd.get(cat_url)
    cats_df.loc[cats_idx] = dog_info_scraping(cat_url, wd)
    cats_idx += 1

  return cats_df

In [ ]:
cats_df = cat_url_scraping()

In [ ]:
cats_df

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,벵갈 캣,암컷,6살,이름(버디),2022-8-26,서울특별시 용산구 유엔빌리지 힐미드빌라,4키로 정도 나가고 소심하고 겁이 많습니다 호피무늬,http://www.angel.or.kr/report/cat/upImg/166182...
1,코리아 쇼트 헤어,암컷,2살,이름(김만두),2022-8-27,인천광역시 계양구 귤현동 굴재길 3-3 현대렉스힐 부근,"삼색 고양이 보는 사람 기준 코 왼쪽에 검정 무늬, 왼쪽 다리 옆에 갈색 무늬, 오...",http://www.angel.or.kr/report/cat/upImg/166159...
2,코리아 쇼트 헤어,암컷,4살,이름(김애용),2022-08-15,경상북도 구미시 지산동 금오여자고등학교 맞은 편 논 부근 에서 실종,코리안 숏헤어 치즈냥 오른쪽 눈이 없고 얼굴에 아아라인처럼 눈선을 따라 진한 갈색 ...,http://www.angel.or.kr/report/cat/upImg/166114...


## `URL_스크래핑` : 그외 동물

In [ ]:
def other_url_scraping():
  wd = webdriver.Chrome('chromedriver', options= chrome_options)
  wd.implicitly_wait(3)
  others_url = 'http://www.angel.or.kr/index.php?code=other'

  wd.get(others_url)

  emergency_others = emergency_scraping(others_url, wd)

  others_df = pd.DataFrame(columns=("Find", "Gender", "Age", "Name", "MissingDay", "Location", "Description", "IMG"))

  others_idx = 0
  for other_url in emergency_others:
    wd.get(other_url)
    others_df.loc[others_idx] = dog_info_scraping(other_url, wd)
    others_idx += 1

  missing_others_url = missing_animals_url('other', wd)

  for other_url in missing_others_url:
    wd.get(other_url)
    others_df.loc[others_idx] = dog_info_scraping(other_url, wd)
    others_idx += 1
  return others_df


In [ ]:
others_df = other_url_scraping()

In [ ]:
others_df

,Find,Gender,Age,Name,MissingDay,Location,Description,IMG
0,앵무새,암컷,1살,이름(일랑이),2022-8-22,대구광역시 수성구 대구 범어동 수성구청역점 서브웨이 뒷쪽 주택단지,블루 퀘이커 앵무새 비둘기보다 많이 작고 참새보다 커요 윙컷을 안한 상태이고 부리쪽...,http://www.angel.or.kr/report/other/upImg/1661...
1,앵무새,암컷,1살,이름(찬송이),2022-8-12,제주특별자치도 제주시 연동 한라유치원부근 산책로,초록색퀘이커예요 빵.국수좋아해요 겁이많아서 물수도있어요,http://www.angel.or.kr/report/other/upImg/1660...
2,앵무새,수컷,1살미만,이름(산초),2022-08-04,"전라북도 군산시 장자도에서 실종. 선유도,무녀도,신시도 발견가능성 높음","회색앵무, 발목링을 하고있습니다. 큰소릴 내거나 섣불리 잡으려 마시고 빠른 연락부탁...",http://www.angel.or.kr/report/other/upImg/1659...
